In [72]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import pandas as pd
from sklearn.model_selection import train_test_split

In [73]:
#Helper functions for data processing
def card_to_num(card):
    raw_rank = card[:-1]
    
    ranks = {
        '2' : 0,
        '3' : 1,
        '4' : 2, 
        '5' : 3,
        '6' : 4, 
        '7' : 5, 
        '8' : 6, 
        '9' : 7, 
        '10': 8, 
        'J' : 9, 
        'Q' : 10, 
        'K' : 11, 
        'A': 12
    }

    return ranks[raw_rank]

def hand_to_list(hand):
    '''Takes hand like KH-AC and outputs list of card numbers'''
    hand_list_1 = hand.split("-")
    hand_list_2 = [card_to_num(card) for card in hand_list_1]
    return hand_list_2

result_mapping = {
    'hit' : 0,
    'stand' : 1,
    'double down' : 2
}

batch_size = 32

# Defining Dataset Class
class Blackjack_Dataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [78]:
split_or_not_raw_df = pd.read_csv('CSVs/split_or_not.csv')

In [79]:
sn_train_df, sn_test_df = train_test_split(split_or_not_raw_df, test_size=0.2)

def clean_up(dataframe_raw):
    # Cleaned split_or_not
    dataframe_raw['dealer_upcard'] = dataframe_raw['dealer_upcard'].apply(card_to_num)
    dataframe_raw['player_hand'] = dataframe_raw['player_hand'].apply(hand_to_list)
    dataframe_raw['player_hand'] = dataframe_raw['player_hand'].apply(lambda hand: hand[0])
    dataframe_clean = dataframe_raw.rename(columns = {'player_hand':'player_upcard'})

    # Turning into tensor matrices
    # split_or_not
    x1 = torch.tensor(dataframe_clean['player_upcard'].values, dtype=torch.float32).unsqueeze(1)
    x2 = torch.tensor(dataframe_clean['dealer_upcard'].values, dtype=torch.float32).unsqueeze(1)
    y = torch.tensor(dataframe_clean['result'].values, dtype=torch.float32).view(-1,1)

    X = torch.cat([x1,x2], dim=1)

    return Blackjack_Dataset(X,y)

sn_train_dataset = clean_up(sn_train_df)
sn_test_dataset = clean_up(sn_test_df)

sn_train_dataloader = DataLoader(sn_train_dataset, batch_size=batch_size, shuffle=True)
sn_test_dataloader = DataLoader(sn_test_dataset, batch_size=batch_size, shuffle=True)

In [80]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()

    for batch, (X,y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Printing Training Update on every 100th batch
        if (batch + 1) % 100 == 0: 
            loss = loss.item()
            current = batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    #Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    #Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True

    with torch.no_grad():
        for X, y in dataloader: 
            pred = model(X)
            probs = torch.sigmoid(pred)
            preds = (probs >= 0.5).float()
            correct += (preds.view(-1) == y.view(-1)).float().sum()
            test_loss += loss_fn(pred, y).item()


    test_loss /= num_batches
    correct /= size 
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [81]:
class sn_NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

sn_model = sn_NeuralNetwork()

learning_rate = 0.0005 
epochs = 20

loss_fn = nn.BCEWithLogitsLoss()
sn_optimizer = torch.optim.SGD(sn_model.parameters(), lr=learning_rate)

In [82]:
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------------")
    train_loop(sn_train_dataloader, sn_model, loss_fn, sn_optimizer)
    test_loop(sn_test_dataloader, sn_model, loss_fn)
print("Done!")

Epoch 1
---------------------------
loss: 0.623670  [ 3200/141921]
loss: 0.606927  [ 6400/141921]
loss: 0.584664  [ 9600/141921]
loss: 0.603284  [12800/141921]
loss: 0.537882  [16000/141921]
loss: 0.697550  [19200/141921]
loss: 0.609404  [22400/141921]
loss: 0.706480  [25600/141921]
loss: 0.667958  [28800/141921]
loss: 0.587952  [32000/141921]
loss: 0.626983  [35200/141921]
loss: 0.640690  [38400/141921]
loss: 0.606481  [41600/141921]
loss: 0.606611  [44800/141921]
loss: 0.639164  [48000/141921]
loss: 0.627495  [51200/141921]
loss: 0.614264  [54400/141921]
loss: 0.676484  [57600/141921]
loss: 0.564499  [60800/141921]
loss: 0.506050  [64000/141921]
loss: 0.539915  [67200/141921]
loss: 0.518879  [70400/141921]
loss: 0.814473  [73600/141921]
loss: 0.640977  [76800/141921]
loss: 0.466473  [80000/141921]
loss: 0.631725  [83200/141921]
loss: 0.636706  [86400/141921]
loss: 0.597137  [89600/141921]
loss: 0.629900  [92800/141921]
loss: 0.558068  [96000/141921]
loss: 0.620879  [99200/141921]
los